In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('..'))
from torch.utils.data import DataLoader, Subset
from pkldataset import PKLDataset

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark   = False

class CNN(nn.Module):
    def __init__(self, input_length: int = 2800, num_classes: int = 10, input_channels: int = 1):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=31, padding=15),
            nn.BatchNorm1d(16), nn.ReLU(inplace=True), nn.MaxPool1d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=31, padding=15),
            nn.BatchNorm1d(32), nn.ReLU(inplace=True), nn.MaxPool1d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=31, padding=15),
            nn.BatchNorm1d(64), nn.ReLU(inplace=True), nn.MaxPool1d(2)
        )
        conv_output_length = input_length // 8
        self.feature_extractor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * conv_output_length, 128),
            nn.ReLU(inplace=True)
        )
        self.classifier_head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.feature_extractor(x)
        x = self.classifier_head(x)
        return x

    def extract_features(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return self.feature_extractor(x)

def train_model(model, train_loader, criterion, optimizer, scheduler,
                num_epochs, device, max_grad_norm=1.0):
    best_loss, best_state = float('inf'), None
    for epoch in range(num_epochs):
        model.train()
        running = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            if targets.dim() > 1:
                targets = targets.argmax(dim=1)
            optimizer.zero_grad()
            loss = criterion(model(inputs), targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            running += loss.item() * inputs.size(0)
        scheduler.step()
        epoch_loss = running / len(train_loader.dataset)
        if epoch_loss < best_loss:
            best_loss, best_state = epoch_loss, model.state_dict()
    model.load_state_dict(best_state)
    return model

def eval_model(model, loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            if targets.dim() > 1:
                targets = targets.argmax(dim=1)
            _, pred = model(inputs).max(1)
            total   += targets.size(0)
            correct += (pred == targets).sum().item()
    return 100. * correct / total

# === CONFIG ===
train_path_1  = r"C:\Users\gus07\Desktop\data hiwi\preprocessing\DC\T197\RP"
transfer_sets = ["../datasets/RPDC185/train_500"]
val_paths = [
    "../datasets/RPDC185/val_100",
    "../datasets/RPDC188/val_1000",
    "../datasets/RPDC191/val_1000",
    "../datasets/RPDC194/val_1000",
    "../datasets/RPDC197/val_1000",
]
seeds         = [101, 202, 303, 404, 505, 606, 707, 808, 909, 1001]
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion     = nn.CrossEntropyLoss()

# ──────────────────────────────────────────────

# 1) ONE‐TIME SOURCE TRAINING (fixed seed)
# ──────────────────────────────────────────────
seed0 = 42
set_seed(seed0)
# split & loaders
train_ds, val_ds = PKLDataset.split_dataset(train_path_1)
loader_tr = DataLoader(train_ds, batch_size=64, shuffle=True)
loader_va = DataLoader(val_ds,   batch_size=64, shuffle=False)

# instantiate, train, evaluate
source_model = CNN().to(device)
opt0   = optim.Adam(source_model.parameters(), lr=1e-3, weight_decay=1e-5)
sch0   = optim.lr_scheduler.StepLR(opt0, step_size=50, gamma=0.1)
source_model = train_model(source_model, loader_tr, criterion, opt0, sch0, num_epochs=10, device=device)
print(f"Source (phase1) val-acc: {eval_model(source_model, loader_va, device):.2f}%")

# save once
pretrained_state = source_model.state_dict()


results = {t:{vp:[] for vp in val_paths} for t in transfer_sets}

for seed in seeds:
    print(f"\n>>> Adaptation seed {seed}")
    set_seed(seed)

    for t in transfer_sets:
        # reload student from the single pretrained source
        student = CNN().to(device)
        student.load_state_dict(pretrained_state)
        student.train()

        # Freeze head initially
        for p in student.classifier_head.parameters():
            p.requires_grad = False
        feat_params = (
            list(student.conv1.parameters()) +
            list(student.conv2.parameters()) +
            list(student.conv3.parameters()) +
            list(student.feature_extractor.parameters())
        )

        shot_optimizer = optim.Adam(feat_params, lr=1e-4, weight_decay=1e-5)
        # smooth cosine decay over ALL epochs
        total_epochs   = 100
        shot_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            shot_optimizer, T_max=total_epochs, eta_min=1e-6
        )

        lambda_shot    = 1.0
        im_only        = 50
        pseudo_labels_all = None
        pseudo_confidence = None

        dataset_t      = PKLDataset(t)
        unlab_loader   = DataLoader(dataset_t, batch_size=64, shuffle=True)
        full_loader_t  = DataLoader(dataset_t, batch_size=64, shuffle=False)

        for epoch in range(total_epochs):
            student.train()

            # --- Information Maximization ---
            if epoch < im_only:
                running_Hc = running_Hm = 0.0
                for x, _ in unlab_loader:
                    x = x.to(device)
                    logits = student(x)
                    probs  = torch.softmax(logits, dim=1)
                    logp   = torch.log(probs + 1e-12)
                    Hc     = -(probs * logp).sum(1).mean()
                    p_bar  = probs.mean(0)
                    Hm     = -(p_bar * torch.log(p_bar + 1e-12)).sum()
                    loss = Hc - lambda_shot * Hm

                    shot_optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(feat_params, 1.0)
                    shot_optimizer.step()

                    running_Hc += Hc.item() * x.size(0)
                    running_Hm += Hm.item() * x.size(0)

                if (epoch+1)%20==0 or epoch==0:
                    N = len(unlab_loader.dataset)
                    print(f"  [IM] Epoch {epoch+1}/{total_epochs}  "
                          f"H_cond={(running_Hc/N):.4f}  H_marg={(running_Hm/N):.4f}")

                shot_scheduler.step()

                # compute pseudo‐labels at end of IM
                if epoch == im_only - 1:
                    student.eval()
                    feats, probs_all = [], []
                    with torch.no_grad():
                        for x, _ in full_loader_t:
                            x = x.to(device)
                            f = student.extract_features(x)
                            p = torch.softmax(student.classifier_head(f), dim=1)
                            feats.append(f.cpu())
                            probs_all.append(p.cpu())
                    feats = torch.cat(feats)
                    probs_all = torch.cat(probs_all)

                    # centroids & hard assignments
                    C = probs_all.size(1)
                    centroids = torch.zeros(C, feats.size(1))
                    for k in range(C):
                        w = probs_all[:,k].unsqueeze(1)
                        denom = w.sum()
                        if denom>0:
                            centroids[k] = (w*feats).sum(0)/denom
                    dists = torch.cdist(feats, centroids)
                    pseudo_labels_all  = torch.argmin(dists, dim=1)
                    pseudo_confidence = probs_all.max(1).values

                    # unfreeze head & add it to optimizer with lower LR
                    for p in student.classifier_head.parameters():
                        p.requires_grad = True
                    shot_optimizer.add_param_group({
                        'params': student.classifier_head.parameters(),
                        'lr': shot_optimizer.param_groups[0]['lr'] * 0.2,
                        'weight_decay': 1e-5
                    })
                    student.train()

            # --- Pseudo‐label Fine‐tuning ---
            else:
                if pseudo_labels_all is None:
                    raise RuntimeError("No pseudo-labels!")
                running_pl = 0.0
                # shuffle indices each epoch
                idxs = torch.randperm(len(dataset_t)).tolist()
                pl_subset = Subset(dataset_t, idxs)
                pl_loader = DataLoader(pl_subset, batch_size=64, shuffle=False)

                for b_idx, (x, _) in enumerate(pl_loader):
                    start = b_idx * pl_loader.batch_size
                    end   = start + x.size(0)
                    batch_idx = idxs[start:end]
                    conf = pseudo_confidence[batch_idx].to(device)
                    labels = pseudo_labels_all[batch_idx].to(device)

                    x = x.to(device)
                    f = student.extract_features(x)
                    logits = student.classifier_head(f)

                    # confidence‐weighted loss
                    per_sample = nn.CrossEntropyLoss(reduction='none')(logits, labels)
                    # optional thresholding to skip very low‐conf samples
                    mask = conf >= 0.3
                    if mask.sum()>0:
                        loss = (per_sample[mask] * conf[mask]).mean()
                    else:
                        loss = per_sample.mean()

                    shot_optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(shot_optimizer.param_groups[0]['params'], 1.0)
                    shot_optimizer.step()

                    running_pl += loss.item() * x.size(0)

                if (epoch+1)%20==0 or epoch==im_only:
                    N = len(dataset_t)
                    print(f"  [PL] Epoch {epoch+1}/{total_epochs}  Pseudo‐CE Loss: {(running_pl/N):.4f}")

                shot_scheduler.step()

        # evaluate on each target val set
        student.eval()
        for vp in val_paths:
            vl = DataLoader(PKLDataset(vp), batch_size=64, shuffle=False)
            acc = eval_model(student, vl, device)
            results[t][vp].append(acc)
            print(f"  → {vp}: {acc:.2f}%")

# final summary
print("\n=== Mean ± Std Dev over seeds ===")
for t in transfer_sets:
    for vp in val_paths:
        arr = np.array(results[t][vp])
        print(f"{t} → {vp}: mean={arr.mean():.2f}%, std={arr.std(ddof=1):.2f}%")

Source (phase1) val-acc: 99.30%

>>> Adaptation seed 101
  [IM] Epoch 1/100  H_cond=0.1773  H_marg=2.0991
  [IM] Epoch 20/100  H_cond=0.0476  H_marg=2.2159
  [IM] Epoch 40/100  H_cond=0.0337  H_marg=2.2428
  [PL] Epoch 51/100  Pseudo‐CE Loss: 0.2762
  [PL] Epoch 60/100  Pseudo‐CE Loss: 0.2229
  [PL] Epoch 80/100  Pseudo‐CE Loss: 0.0623
  [PL] Epoch 100/100  Pseudo‐CE Loss: 0.0884
  → ../datasets/RPDC185/val_100: 95.30%
  → ../datasets/RPDC188/val_1000: 98.80%
  → ../datasets/RPDC191/val_1000: 91.91%
  → ../datasets/RPDC194/val_1000: 88.30%
  → ../datasets/RPDC197/val_1000: 88.30%

>>> Adaptation seed 202
  [IM] Epoch 1/100  H_cond=0.1871  H_marg=2.0612
  [IM] Epoch 20/100  H_cond=0.0629  H_marg=2.2284
  [IM] Epoch 40/100  H_cond=0.0253  H_marg=2.2397
  [PL] Epoch 51/100  Pseudo‐CE Loss: 0.1884
  [PL] Epoch 60/100  Pseudo‐CE Loss: 0.0966
  [PL] Epoch 80/100  Pseudo‐CE Loss: 0.0575
  [PL] Epoch 100/100  Pseudo‐CE Loss: 0.0305
  → ../datasets/RPDC185/val_100: 95.50%
  → ../datasets/RPDC18